Most up to date top_offender approach, level 1

In [1]:
# %load_ext autoreload
# %reload_ext autoreload
# %autoreload 2

In [2]:
import os
os.chdir('C:\\Users\\WDG1DCR\\Desktop\\my_projects\\data_analysis\\data_cleanse')
# os.chdir('C:\\Users\\gratten\\Documents\\python\\projects\\data_cleanse')

import pandas as pd
from prep import filterx
from prep import sort
from plot import bar
from utility import generate

ImportError: cannot import name 'filter' from 'prep' (C:\Users\WDG1DCR\Desktop\my_projects\data_analysis\data_cleanse\prep\__init__.py)

In [ ]:
fileName = 'filelist_8_20'
df_og = pd.read_csv('datasets/' + fileName + '.csv', sep=';')

In [ ]:
df = filterx.sld(df_og)

In [ ]:
dups = df[df.duplicated(['name_low'], keep=False)]
dups = dups.sort_values(['name_low'])

In [ ]:
dups = dups.reset_index()

In [ ]:
dups.head()

In [ ]:
# dups['path'].value_counts()

In [ ]:
top_offenders = dups['path'].value_counts().index.to_list()

In [ ]:
len(top_offenders)

In [ ]:
def build_data(dups, num):
    directory = top_offenders[num]
    
    file_list = []
    for i in range(len(dups['path'])):
        if dups['path'][i] == top_offenders[num]:
            file_list.append(dups['name_low'].iloc[i])
            
    file_paths = {}
    for file in file_list:
        dups_trimmed = dups[(dups['name_low'] == file)]
        path_list = []
        for path in dups_trimmed['path']:
            path_list.append(path)
        file_paths[file] = path_list
        
    totalFiles = 0
    for k, v in file_paths.items():
        totalFiles += len(v)
    
    occur_list = []
    for k, v in file_paths.items():
        if len(v) not in occur_list:
            occur_list.append(len(v))
    
    occur_dict = {}
    for i in occur_list:
        occur_dict[i] = 0
        for v in file_paths.values():
            if i == len(v):
                occur_dict[i] += 1
                
    num_list = []
    occur_list = []
    for i in range(len(occur_dict)):
        key = min(occur_dict.keys())
        num_list.append(key)
        value = occur_dict.pop(key)
        occur_list.append(value)    
        
#     print(totalFiles)
        
    return num_list, occur_list, totalFiles, file_paths

In [ ]:
def build_data2(offender, file_paths, quant):
    
    file_paths_trimmed = {}
    for k, v in file_paths.items():
        if offender in v:
            if len(v) == quant: 
                file_paths_trimmed[k] = v
                
    path_list = []
    for v in file_paths_trimmed.values():
        for i in v:
            path_list.append(i)
            
    path_list = pd.Series(path_list)
    
    occurence_list = path_list.value_counts().to_list()
    
    path_list = path_list.value_counts().index.to_list()
    
    return occurence_list, path_list, file_paths_trimmed

In [ ]:
def batch2(files_to_move):
    os.chdir('H:/temp/data_cleanse_batch_files')
    moveFileName = '8_19_files_to_remove.txt'
    moveFile = open(moveFileName, 'w')
    for i in files_to_move:
        newLoc_path = i
        newLoc_path = newLoc_path[7:]
        newLoc_path = newLoc_path[:newLoc_path.rindex('\\')]
        newLoc_path = newLoc_path.replace(' ', '')
        newLoc = 'C:\\Users\\wdg1dcr\\Desktop\\dupBackup\\movedFiles2\\' + newLoc_path
        moveFile.write(f'md {newLoc} 2> nul & move \"{i}\"*.* {newLoc}\n')
    moveFile.close()
    os.chdir('C:\\Users\\WDG1DCR\\Desktop\\my_projects\\data_analysis\\data_cleanse')

In [ ]:
def chart_em(num_list, occur_list, x):
    
    df = pd.DataFrame({'other paths':num_list, 'occurences':occur_list})
    title = f'{top_offenders[i]}\nTotal files: {totalFiles_list[i]}\n Chart {x}'
    ax = df.plot.bar(x='other paths', y='occurences', rot=0, figsize=(10, 5), title = title)

In [ ]:
def chart_em2(list1, list2):
    df = pd.DataFrame({'other paths':list1, 'occurences':list2})
#     title = f'{titleA}'
    ax = df.plot.barh(x='other paths', y='occurences', rot=0, figsize=(3, 7))

In [ ]:
############################################
# limit for how many charts to generate
############################################
my_range = 20

num_list_list = []
occur_list_list = []
totalFiles_list = []
file_paths_list = []

for i in range(my_range):
    outputA, outputB, totalFiles, file_paths = build_data(dups, i)
    num_list_list.append(outputA)
    occur_list_list.append(outputB)
    totalFiles_list.append(totalFiles)
    file_paths_list.append(file_paths)

In [ ]:
for i in range(len(num_list_list)):
    chart_em(num_list_list[i], occur_list_list[i], i)

In [ ]:
############################################
# choose chart num and column for file list
############################################
chart_num = 10
quant = 2

In [ ]:
occurence_list, path_list, file_paths_trimmed = build_data2(top_offenders[chart_num], file_paths_list[chart_num], quant)

In [ ]:
chart_em2(path_list, occurence_list)

In [ ]:
############################################
# choose path to maintain, all others will be deleted
############################################
choose_path2 = 0

print(path_list[choose_path2])

file_list = []
for i in path_list:
    if i != path_list[choose_path2]:
        for k, v in file_paths_trimmed.items():
            if i in v:
                file_and_path = i + '\\' + k
                file_list.append(file_and_path)
                
# batch2(file_list)

In [ ]:
for i in path_list:
    print(i)

In [ ]:
# SORT BY FILES SIZE
file_list = []
for k in file_paths_trimmed.keys():
    file_list.append(k)

In [ ]:
dups_trimmed = dups[dups['name_low'].isin(file_list)]

In [ ]:
for i in range(len(dups_trimmed['size'])):    
    value1 = dups_trimmed['size'].iloc[i]
    value1 = value1.replace(',','')
    value1 = int(value1)
    dups_trimmed['size'].iloc[i] = value1

In [ ]:
dups_trimmed = dups_trimmed.sort_values(by=['size'], ascending=False)

In [ ]:
dups_trimmed.reset_index(inplace=True, drop=True)

In [ ]:
# dups_trimmed.head()

In [ ]:
fname = '20345-310-11.sldprt'

dups_trimmed[dups_trimmed['name_low'] == fname]

In [ ]:
for i in dups_trimmed['name_low'].unique():
    print(i)